<a href="https://colab.research.google.com/github/DimAce74/Education/blob/master/module_pandas_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для работы с большим файлом, установим библиотеку

In [ ]:
!pip install polars

Импортируем необходимые библиотеки.

In [ ]:
import pandas as pd
import polars as pl
import plotly.express as px

Подготовим объекты Polars для дальнейшей работы.

In [ ]:
train_q = pl.scan_csv('data/train.csv')
lectures_q = pl.scan_csv('data/lectures.csv')
questions_q = pl.scan_csv('data/questions.csv')

Посмотрим на схему большого файла

In [ ]:
schema = train_q.collect_schema()
print(schema)

Schema({'row_id': Int64, 'timestamp': Int64, 'user_id': Int64, 'content_id': Int64, 'content_type_id': Int64, 'task_container_id': Int64, 'user_answer': Int64, 'answered_correctly': Int64, 'prior_question_elapsed_time': Float64, 'prior_question_had_explanation': Boolean})


Теперь посмотрим на первые 5 элементов.

In [ ]:
train_head = train_q.head(5).collect()

print(train_head)


shape: (5, 10)
┌────────┬───────────┬─────────┬────────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ row_id ┆ timestamp ┆ user_id ┆ content_id ┆ … ┆ user_answe ┆ answered_c ┆ prior_ques ┆ prior_que │
│ ---    ┆ ---       ┆ ---     ┆ ---        ┆   ┆ r          ┆ orrectly   ┆ tion_elaps ┆ stion_had │
│ i64    ┆ i64       ┆ i64     ┆ i64        ┆   ┆ ---        ┆ ---        ┆ ed_time    ┆ _explanat │
│        ┆           ┆         ┆            ┆   ┆ i64        ┆ i64        ┆ ---        ┆ ion       │
│        ┆           ┆         ┆            ┆   ┆            ┆            ┆ f64        ┆ ---       │
│        ┆           ┆         ┆            ┆   ┆            ┆            ┆            ┆ bool      │
╞════════╪═══════════╪═════════╪════════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 0      ┆ 0         ┆ 115     ┆ 5692       ┆ … ┆ 3          ┆ 1          ┆ null       ┆ null      │
│ 1      ┆ 56943     ┆ 115     ┆ 5716       ┆ … ┆ 2          ┆ 1          ┆ 

Дла дальнейшего переиспользования подготовим еще один более узкий объект-выборку, включающий только вопросы, без лекций

In [ ]:
train_q_quest = train_q.filter(
        pl.col('content_type_id') == 0
    )

Посмотрим описание числовых показателей. Булево приведем к числу

In [ ]:
train_full = (train_q_quest.select(
    # выбираем все столбцы, кроме перечисленных
    pl.all().exclude('row_id', 'content_type_id', 'user_id', 'content_id', 'user_answer', 'task_container_id', 'prior_question_had_explanation'),
    # столбец с булевым значением приведем к числовому
    pl.col('prior_question_had_explanation').cast(pl.Float32)
).collect(streaming=True))
# воспользуемся методом вывода описания
print(train_full.describe())

shape: (9, 6)
┌────────────┬──────────────┬─────────────────┬─────────────────┬─────────────────┬────────────────┐
│ statistic  ┆ timestamp    ┆ content_type_id ┆ answered_correc ┆ prior_question_ ┆ prior_question │
│ ---        ┆ ---          ┆ ---             ┆ tly             ┆ elapsed_time    ┆ _had_explanati │
│ str        ┆ f64          ┆ f64             ┆ ---             ┆ ---             ┆ on             │
│            ┆              ┆                 ┆ f64             ┆ f64             ┆ ---            │
│            ┆              ┆                 ┆                 ┆                 ┆ f64            │
╞════════════╪══════════════╪═════════════════╪═════════════════╪═════════════════╪════════════════╡
│ count      ┆ 1.01230332e8 ┆ 1.01230332e8    ┆ 1.01230332e8    ┆ 9.8878794e7     ┆ 1.00837826e8   │
│ null_count ┆ 0.0          ┆ 0.0             ┆ 0.0             ┆ 2.351538e6      ┆ 392506.0       │
│ mean       ┆ 7.7036e9     ┆ 0.019352        ┆ 0.625164        ┆ 25423.81004

Обратим внимание на значерния и разброс временных показателей - это позвволит в дальнейшем при анализе сгладить результаты с осознанной группировкой. Также видим, что в среднем на 66% вопросов получены верные ответы и что в 91% случаев учащиеся знакомились с объяснением.

Посмотрим на первые 5 элементов файлов с информацией о вопросах и лекциях

In [ ]:
lectures_head = lectures_q.head(5).collect()
questions_head = questions_q.head(5).collect()
print(lectures_head)
print(questions_head)

shape: (5, 4)
┌────────────┬─────┬──────┬──────────────────┐
│ lecture_id ┆ tag ┆ part ┆ type_of          │
│ ---        ┆ --- ┆ ---  ┆ ---              │
│ i64        ┆ i64 ┆ i64  ┆ str              │
╞════════════╪═════╪══════╪══════════════════╡
│ 89         ┆ 159 ┆ 5    ┆ concept          │
│ 100        ┆ 70  ┆ 1    ┆ concept          │
│ 185        ┆ 45  ┆ 6    ┆ concept          │
│ 192        ┆ 79  ┆ 5    ┆ solving question │
│ 317        ┆ 156 ┆ 5    ┆ solving question │
└────────────┴─────┴──────┴──────────────────┘
shape: (5, 5)
┌─────────────┬───────────┬────────────────┬──────┬────────────────┐
│ question_id ┆ bundle_id ┆ correct_answer ┆ part ┆ tags           │
│ ---         ┆ ---       ┆ ---            ┆ ---  ┆ ---            │
│ i64         ┆ i64       ┆ i64            ┆ i64  ┆ str            │
╞═════════════╪═══════════╪════════════════╪══════╪════════════════╡
│ 0           ┆ 0         ┆ 0              ┆ 1    ┆ 51 131 162 38  │
│ 1           ┆ 1         ┆ 1            

Посмотрим зависимость правильных ответов от получения разъяснений на предыдущий вопрос.

In [ ]:
print(
    train_q_quest.select(
        pl.col('prior_question_had_explanation'),
        pl.col('answered_correctly'),
    ).drop_nulls()
    .group_by('prior_question_had_explanation')
    .agg(
        pl.col('answered_correctly')
        .mean()
    ).collect(streaming=True)
)


shape: (2, 2)
┌────────────────────────────────┬────────────────────┐
│ prior_question_had_explanation ┆ answered_correctly │
│ ---                            ┆ ---                │
│ bool                           ┆ f64                │
╞════════════════════════════════╪════════════════════╡
│ true                           ┆ 0.673189           │
│ false                          ┆ 0.500629           │
└────────────────────────────────┴────────────────────┘


Видим, что, если учащийся получал разъяснения, то отвечал правильно на следующий вопрос в 67% случаев, иначе только в 50%.

Посмотрим с другой стороны - как часто учащиеся получали разъяснения после правильного/неправильного ответа. Здесь нам придется сдвинуть строки в столбцах, чтобы разъяснение относилось к анализируемому вопросу

In [ ]:
print(
    train_q_quest.select(
        # в столбце с разъяснениями возьмем все элементы, кроме первого
        pl.col('prior_question_had_explanation').slice(1, None),
        # в столбце с ответами возьмем все элементы, кроме последнего
        pl.col('answered_correctly').slice(0, pl.col('answered_correctly').count() - 1)
    # сгруппируем данные по столбцу ответа
    ).group_by('answered_correctly')
    # для стобца факта разъяснений проведем аггрегацию
    .agg(
        pl.col('prior_question_had_explanation')
        # приводим булево к числу
        .cast(pl.Float32)
        # вычисляем среднее
        .mean()
        # так как смысл столбца поменялся, переименуем его
        .alias('this_question_had_explanation')
    # запускаем обработку со стримингом
    ).collect(streaming=True)
)

shape: (2, 2)
┌────────────────────┬───────────────────────────────┐
│ answered_correctly ┆ this_question_had_explanation │
│ ---                ┆ ---                           │
│ i64                ┆ f64                           │
╞════════════════════╪═══════════════════════════════╡
│ 1                  ┆ 0.929161                      │
│ 0                  ┆ 0.86446                       │
└────────────────────┴───────────────────────────────┘


Видим, что после правильного ответа учащиеся чаще (93%) брали разъяснения, чем после неправильного (86%)

Посмотрим зависимость правильности ответов от общего времени

In [ ]:
result = ((train_q_quest.filter(
    # отрезаем выбросы выше 90-го перцентиля
    (pl.col('timestamp') < pl.col('timestamp').quantile(0.9)) &
    # отрезаем выбросы ниже относительно малого значения
    (pl.col('timestamp') > 300000000)
).group_by(
    # группируем данные по времени, сгладив результаты
    (pl.col('timestamp') // 250000000).alias('time'),
).agg(
    # для аггрегации возьмем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
# отсортируем по времени
)).sort('time')
          .collect(streaming=True))

# выведем линейный график
px.line(result, x='time', y='mean', )

Видим, что повышательный тренд есть, но при этом диапозон разброса значений очень небольшой

Для большей информативности посмотрим, как зависит среднее количество правильных ответов от полного времени обучения.

In [ ]:
# группируем данные по пользователям
result = (train_q.group_by(
    pl.col('user_id')
# посчитаем необходимые аггрегаты
).agg(
    # количество строк
    pl.col('row_id').count(),
    # макимальное значение времени
    pl.col('timestamp').max().alias('time'),
    # среднее количество правильных ответов
    pl.col('answered_correctly').mean(),
).filter(
    # уберем из выборки результаты с малым количеством строк
    pl.col('row_id') > pl.col('row_id').quantile(0.1)
).group_by(
    # группируем по сглаженному времени
    pl.col('time') // 1000000000
).agg(
    # аггрегируем среднее по правильным ответам
    pl.col('answered_correctly').mean()
).sort('time')
          .collect(streaming=True))

# строим график
px.line(result, x='time', y='answered_correctly')

На графике хороша видна прямая зависимость количества правильных ответов от общего времени.

Посмотрим на зависимость правильных ответов от затраченного на вопрос времени

In [ ]:
result = ((train_q.filter(
    # отбросим слишком большие значения времени
    (pl.col('prior_question_elapsed_time') < pl.col('prior_question_elapsed_time').quantile(0.95)) &
    # отбросим слишком малые значения времени
    (pl.col('prior_question_elapsed_time') > pl.col('prior_question_elapsed_time').quantile(0.05))
).select(
    # для соответствия столбцов делаем сдвиг
    pl.col('prior_question_elapsed_time').slice(1, None),
    pl.col('answered_correctly').slice(0, pl.col('answered_correctly').count() - 1)
).group_by(
    # группируем по сглаженному до секунд времени
    (pl.col('prior_question_elapsed_time') // 1000).alias('time')
# считаем среднее количество правильных ответов
).agg(pl.col('answered_correctly').mean()))
          .sort(pl.col('time'))
          .collect(streaming=True))

# строим график
px.line(result, x='time', y='answered_correctly')

Видим, что, при времени после 15 секунд, среднее количество правильных ответов снижается

Посмотрим на связь с метаданными по вопросам.
Сначала подготовим объекты выборки

In [ ]:
# джойним датафрэймы с результатами и метаинформацией
quest_joined_q = (train_q_quest.join(questions_q, left_on='content_id', right_on='question_id', how='left'))
# в столбце с тэгами данные преобразовываем в список
quest_joined_split_q = quest_joined_q.with_columns(
    pl.col('tags').str.split(" ")
)

Найдем темы с наибольшим и наименьшим средним количеством правильных ответов.

In [ ]:
# берем из датафрэйма столбцы с тэгами и правильностью ответов
quest_joined_expl_q = (quest_joined_split_q.select(
    pl.col('answered_correctly'),
    pl.col('tags'),
# разбиваем столбец с тэгами на отдельные строки
).explode('tags')
# группируем по тэгу
.group_by(
    pl.col('tags').alias('tag')
).agg(
    # считаем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
).drop_nulls()
# и сортируем по нему
.sort('mean'))

print(quest_joined_expl_q.collect(streaming=True))


shape: (188, 3)
┌──────┬──────────┬────────┐
│ tags ┆ mean     ┆ count  │
│ ---  ┆ ---      ┆ ---    │
│ str  ┆ f64      ┆ u32    │
╞══════╪══════════╪════════╡
│ 24   ┆ 0.385725 ┆ 256613 │
│ 23   ┆ 0.438487 ┆ 465904 │
│ 19   ┆ 0.439938 ┆ 309969 │
│ 151  ┆ 0.493155 ┆ 537180 │
│ 167  ┆ 0.500432 ┆ 341067 │
│ …    ┆ …        ┆ …      │
│ 40   ┆ 0.813804 ┆ 226514 │
│ 149  ┆ 0.848015 ┆ 396973 │
│ 130  ┆ 0.853152 ┆ 243708 │
│ 187  ┆ 0.85602  ┆ 252069 │
│ 68   ┆ 0.863945 ┆ 120378 │
└──────┴──────────┴────────┘


Видим, что на вопросы с тэгом=24 получено всего 38.6% правильных ответов, а на вопросы с тэгом=68 более 86%

Теперь посмотрим на такую же статистику по разделам.

In [ ]:
# берем из датафрэйма столбцы с темами и правильностью ответов
quest_joined_expl_q = (quest_joined_split_q.select(
    pl.col('answered_correctly'),
    pl.col('part'),
# группируем по разделу
).group_by(
    pl.col('part')
).agg(
    # считаем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
).drop_nulls()
                       # и сортируем по нему
                       .sort('mean'))

print(quest_joined_expl_q.collect(streaming=True))

shape: (7, 2)
┌──────┬──────────┐
│ part ┆ mean     │
│ ---  ┆ ---      │
│ i64  ┆ f64      │
╞══════╪══════════╡
│ 5    ┆ 0.610088 │
│ 4    ┆ 0.630998 │
│ 7    ┆ 0.659601 │
│ 6    ┆ 0.669388 │
│ 3    ┆ 0.701456 │
│ 2    ┆ 0.708694 │
│ 1    ┆ 0.745032 │
└──────┴──────────┘


Видим, что на вопросы с темой=5 получено 61% правильных ответов, а на вопросы с темой=1 более 74%


In [ ]:
# группируем данные по пользователям
result = (train_q.group_by('user_id')
    # посчитаем необходимые аггрегаты
.agg(
    # количество строк
    pl.col('row_id').count().alias('rows'),
    # количество лекций
    (pl.col('content_type_id') == 1).sum().alias('lectures'),
    # количество вопросов
    (pl.col('content_type_id') == 0).sum().alias('questions'),
    # количество правильных ответов
    (pl.col('answered_correctly') == 1).sum().alias('correct'),
).filter(
    # уберем из выборки результаты с малым количеством строк
    (pl.col('rows') > pl.col('rows').quantile(0.1)) &
    # Отсечем выбросы с количеством лекций
    (pl.col('lectures') < pl.col('lectures').quantile(0.95))
# отбираем колонки для дальнейшего анализа
).select(
    # количество лекций
    pl.col('lectures'),
    # считаем процент правильных ответов
    (pl.col('correct') / pl.col('questions') * 100).alias('correct_%')
).group_by(
    # группируем по количеству лекций
    pl.col('lectures')
).agg(
    # аггрегируем среднее по правильным ответам
    pl.col('correct_%').mean()
).sort('lectures')
          .collect(streaming=True))
# строим график
px.line(result, x='lectures', y='correct_%')

На графике хорошо видна прямая зависимость оценки от количества посещенных лекций